# Variational Autoencoder

In [1]:

import logging
from functools import partial

from fastai.basics import *
from fastai.learner import Learner
from fastai.callback.all import *
from fastai.callback.all import EarlyStoppingCallback
from fastai.torch_basics import *

import torch

from IPython.display import display

from torch.nn import Sigmoid

import pandas as pd

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

import vaep
import vaep.nb
from vaep.io import datasplits
from vaep.models import ae
import vaep.models as models
import vaep.model
from vaep.analyzers import analyzers


# overwriting Recorder callback with custom plot_loss
from vaep.models import plot_loss
from fastai import learner
learner.Recorder.plot_loss = plot_loss


logger = vaep.logging.setup_logger(logging.getLogger('vaep'))
logger.info(
    "Experiment 03 - Analysis of latent spaces and performance comparisions")

figures = {}  # collection of ax or figures

vaep - INFO     Experiment 03 - Analysis of latent spaces and performance comparisions


In [2]:
# catch passed parameters
args = None
args = dict(globals()).keys()

Papermill script parameters:

In [3]:
# files and folders
# Datasplit folder with data for experiment
folder_experiment: str = 'runs/example'
folder_data: str = ''  # specify data directory if needed
file_format: str = 'csv'  # file format of create splits, default pickle (pkl)
# Machine parsed metadata from rawfile workflow
fn_rawfile_metadata: str = 'data/dev_datasets/HeLa_6070/files_selected_metadata_N50.csv'
# training
epochs_max: int = 50  # Maximum number of epochs
batch_size: int = 64  # Batch size for training (and evaluation)
cuda: bool = True  # Whether to use a GPU for training
# model
# Dimensionality of encoding dimension (latent space of model)
latent_dim: int = 25
# A underscore separated string of layers, '256_128' for the encoder, reverse will be use for decoder
hidden_layers: str = '256_128'
# force_train:bool = True # Force training when saved model could be used. Per default re-train model
patience: int = 50  # Patience for early stopping
sample_idx_position: int = 0  # position of index which is sample ID
model: str = 'VAE'  # model name
model_key: str = 'VAE'  # potentially alternative key for model (grid search)
save_pred_real_na: bool = True  # Save all predictions for missing values
# metadata -> defaults for metadata extracted from machine data
meta_date_col: str = None  # date column in meta data
meta_cat_col: str = None  # category column in meta data

In [4]:
# Parameters
folder_experiment = "runs/example"
file_format = "csv"
latent_dim = 25
batch_size = 10
epochs_max = 5
hidden_layers = "512_256"
sample_idx_position = 0
cuda = False
save_pred_real_na = True
fn_rawfile_metadata = "data/dev_datasets/HeLa_6070/files_selected_metadata_N50.csv"
model_key = "VAE"


Some argument transformations

In [5]:
args = vaep.nb.get_params(args, globals=globals())
args

{'folder_experiment': 'runs/example',
 'folder_data': '',
 'file_format': 'csv',
 'fn_rawfile_metadata': 'data/dev_datasets/HeLa_6070/files_selected_metadata_N50.csv',
 'epochs_max': 5,
 'batch_size': 10,
 'cuda': False,
 'latent_dim': 25,
 'hidden_layers': '512_256',
 'patience': 50,
 'sample_idx_position': 0,
 'model': 'VAE',
 'model_key': 'VAE',
 'save_pred_real_na': True,
 'meta_date_col': None,
 'meta_cat_col': None}

In [6]:
args = vaep.nb.args_from_dict(args)

if isinstance(args.hidden_layers, str):
    args.overwrite_entry("hidden_layers", [int(x)
                         for x in args.hidden_layers.split('_')])
else:
    raise ValueError(
        f"hidden_layers is of unknown type {type(args.hidden_layers)}")
args

{'batch_size': 10,
 'cuda': False,
 'data': Path('runs/example/data'),
 'epochs_max': 5,
 'file_format': 'csv',
 'fn_rawfile_metadata': 'data/dev_datasets/HeLa_6070/files_selected_metadata_N50.csv',
 'folder_data': '',
 'folder_experiment': Path('runs/example'),
 'hidden_layers': [512, 256],
 'latent_dim': 25,
 'meta_cat_col': None,
 'meta_date_col': None,
 'model': 'VAE',
 'model_key': 'VAE',
 'out_figures': Path('runs/example/figures'),
 'out_folder': Path('runs/example'),
 'out_metrics': Path('runs/example'),
 'out_models': Path('runs/example'),
 'out_preds': Path('runs/example/preds'),
 'patience': 50,
 'sample_idx_position': 0,
 'save_pred_real_na': True}

Some naming conventions

In [7]:
TEMPLATE_MODEL_PARAMS = 'model_params_{}.json'

## Load data in long format

In [8]:
data = datasplits.DataSplits.from_folder(
    args.data, file_format=args.file_format)

vaep.io.datasplits - INFO     Loaded 'train_X' from file: runs/example/data/train_X.csv


vaep.io.datasplits - INFO     Loaded 'val_y' from file: runs/example/data/val_y.csv


vaep.io.datasplits - INFO     Loaded 'test_y' from file: runs/example/data/test_y.csv


data is loaded in long format

In [9]:
data.train_X.sample(5)

Sample ID                                       Gene Names
2020_02_12_05_06_Q-Exactive-HF-X-Orbitrap_6070  COASY        31.968
2020_05_26_14_20_Q-Exactive-HF-X-Orbitrap_6070  FHL3         26.119
2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070  PSMD8        30.127
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070  TRMT5        24.450
2020_02_12_05_06_Q-Exactive-HF-X-Orbitrap_6070  ATP13A1      28.687
Name: intensity, dtype: float64

Infer index names from long format

In [10]:
index_columns = list(data.train_X.index.names)
sample_id = index_columns.pop(args.sample_idx_position)
if len(index_columns) == 1:
    index_column = index_columns.pop()
    index_columns = None
    logger.info(f"{sample_id = }, single feature: {index_column = }")
else:
    logger.info(f"{sample_id = }, multiple features: {index_columns = }")

if not index_columns:
    index_columns = [sample_id, index_column]
else:
    raise NotImplementedError(
        "More than one feature: Needs to be implemented. see above logging output.")

vaep - INFO     sample_id = 'Sample ID', single feature: index_column = 'Gene Names'


load meta data for splits

In [11]:
if args.fn_rawfile_metadata:
    df_meta = pd.read_csv(args.fn_rawfile_metadata, index_col=0)
    display(df_meta.loc[data.train_X.index.levels[0]])
else:
    df_meta = None

Version Content Creation Date  \
Sample ID                                                                       
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070       66   2019-12-18 14:35:26   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070       66   2019-12-19 19:48:33   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070       66   2019-12-20 14:15:00   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070       66   2019-12-27 12:29:44   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070       66   2019-12-29 15:06:50   
2019_12_29_18_18_Q-Exactive-HF-X-Orbitrap_6070       66   2019-12-29 18:18:31   
2020_01_02_17_38_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-02 17:38:26   
2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-03 11:17:32   
2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-03 16:58:20   
2020_01_03_20_10_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-03 20:10:03   
2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-04 04:23:07   
2020_01_04_10_03_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-04 10:03:55   
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-04 14:59:27   
2020_01_06_20_17_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-06 20:17:34   
2020_01_08_16_43_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-08 16:43:19   
2020_01_09_11_07_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-09 11:07:52   
2020_01_15_13_56_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-15 13:56:57   
2020_01_20_15_10_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-20 15:10:06   
2020_02_05_20_55_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-05 20:55:39   
2020_02_10_15_41_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-10 15:41:47   
2020_02_11_10_35_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-11 10:35:29   
2020_02_12_05_06_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-12 05:06:18   
2020_02_13_00_26_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-13 00:26:52   
2020_02_13_03_11_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-13 03:11:08   
2020_02_17_13_55_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-17 13:55:21   
2020_02_18_01_25_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-18 01:25:20   
2020_02_18_18_55_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-18 18:55:51   
2020_02_28_12_27_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-28 12:27:07   
2020_03_01_23_00_Q-Exactive-HF-X-Orbitrap_6070       66   2020-03-01 23:00:51   
2020_03_06_16_22_Q-Exactive-HF-X-Orbitrap_6070       66   2020-03-06 16:22:33   
2020_03_07_18_15_Q-Exactive-HF-X-Orbitrap_6070       66   2020-03-07 18:15:03   
2020_03_11_11_25_Q-Exactive-HF-X-Orbitrap_6070       66   2020-03-11 11:25:29   
2020_05_04_11_39_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-04 11:39:52   
2020_05_12_15_13_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-12 15:13:44   
2020_05_12_18_10_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-12 18:10:40   
2020_05_12_21_07_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-12 21:07:35   
2020_05_14_14_46_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-14 14:46:12   
2020_05_14_17_28_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-14 17:28:58   
2020_05_14_20_11_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-14 20:11:31   
2020_05_15_10_30_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-15 10:30:10   
2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-20 12:33:40   
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-20 15:35:03   
2020_05_22_14_57_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-22 14:57:09   
2020_05_22_17_43_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-22 17:43:19   
2020_05_26_14_20_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-26 14:20:17   
2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-27 13:57:57   
2020_05_28_04_06_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-28 04:06:33   
2020_06_01_10_22_Q-Exactive-HF-X-Orbitrap_6070       66   2020-06-01 10:22:06  

## Initialize Comparison

- replicates idea for truely missing values: Define truth as by using n=3 replicates to impute
  each sample
- real test data:
    - Not used for predictions or early stopping.
    - [x] add some additional NAs based on distribution of data

In [12]:
freq_feat = vaep.io.datasplits.load_freq(args.data)
freq_feat.head()  # training data

Gene Names
AAR2     45
ABCB6    35
ABHD11   30
ACAD9    50
ACTL6A   49
Name: freq, dtype: int64

### Produce some addional simulated samples

The validation simulated NA is used to by all models to evaluate training performance.

In [13]:
val_pred_simulated_na = data.val_y.to_frame(name='observed')
val_pred_simulated_na

observed
Sample ID                                      Gene Names          
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 GEMIN5        30.946
2020_02_28_12_27_Q-Exactive-HF-X-Orbitrap_6070 AAR2          27.248
2020_02_13_03_11_Q-Exactive-HF-X-Orbitrap_6070 ADAM10        29.310
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070 NDUFAF4       24.352
2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070 EPN1          29.250
...                                                             ...
2020_03_01_23_00_Q-Exactive-HF-X-Orbitrap_6070 PRRC2B        24.294
2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070 ARGLU1        28.471
2020_02_18_18_55_Q-Exactive-HF-X-Orbitrap_6070 GBF1          29.470
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 DENR          27.902
2020_05_12_21_07_Q-Exactive-HF-X-Orbitrap_6070 SNRPF         27.487

[859 rows x 1 columns]

In [14]:
test_pred_simulated_na = data.test_y.to_frame(name='observed')
test_pred_simulated_na.describe()

observed
count   859.000
mean     28.270
std       2.543
min      22.254
25%      26.477
50%      28.068
75%      30.064
max      35.919

## Data in wide format

- Autoencoder need data in wide format

In [15]:
data.to_wide_format()
args.M = data.train_X.shape[-1]
data.train_X.head()

Gene Names                                       AAR2  ABCB6  ABHD11  ACAD9  \
Sample ID                                                                     
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 27.248 24.472  25.714 28.542   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070 27.211 23.775     NaN 28.437   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070 27.377 26.852  26.046 28.907   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070    NaN 24.047  24.788 27.894   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070 26.520    NaN     NaN    NaN   

Gene Names                                      ACTL6A  ACTN1  ACTR3  ADAM10  \
Sample ID                                                                      
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070  29.335 30.837 29.713  26.702   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070  29.692 30.635 30.340  25.929   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070  30.369 31.252 30.715  26.403   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070  29.359 29.996 29.686     NaN   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070  30.164 33.143 30.985  25.824   

Gene Names                                      AHSA1    AK3  ...  WBSCR16  \
Sample ID                                                     ...            
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 31.696 26.337  ...   26.360   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070 31.296 25.415  ...      NaN   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070    NaN 25.554  ...   27.147   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070 30.882 22.498  ...      NaN   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070 31.145 29.210  ...      NaN   

Gene Names                                      WDR18  WDR33  WDR36  WDR61  \
Sample ID                                                                    
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 28.258 25.507 30.009 28.345   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070    NaN 23.620 29.348 28.728   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070 28.335    NaN 30.563 28.859   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070 28.228    NaN 29.497 27.716   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070    NaN 26.818 29.374 29.407   

Gene Names                                       WNK1   YBX1  YTHDF1  YWHAH  \
Sample ID                                                                     
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 27.701 32.374  24.925 29.609   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN 29.027   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070 28.405 32.328  25.498 30.170   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070 27.349 31.794  24.701 29.232   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070 28.900 32.443     NaN 30.615   

Gene Names                                      ZNF326  
Sample ID                                               
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070  28.756  
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070  28.393  
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070  29.208  
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070  28.139  
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070  29.189  

[5 rows x 376 columns]

### Add interpolation performance

### Fill Validation data with potentially missing features

In [16]:
data.train_X

Gene Names                                       AAR2  ABCB6  ABHD11  ACAD9  \
Sample ID                                                                     
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 27.248 24.472  25.714 28.542   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070 27.211 23.775     NaN 28.437   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070 27.377 26.852  26.046 28.907   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070    NaN 24.047  24.788 27.894   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070 26.520    NaN     NaN    NaN   
2019_12_29_18_18_Q-Exactive-HF-X-Orbitrap_6070 26.917    NaN     NaN    NaN   
2020_01_02_17_38_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN 27.112   
2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070 24.846    NaN     NaN 27.670   
2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070 25.971    NaN     NaN 28.528   
2020_01_03_20_10_Q-Exactive-HF-X-Orbitrap_6070 26.897    NaN     NaN 28.555   
2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070 27.828    NaN     NaN 29.198   
2020_01_04_10_03_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN 24.812   
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_01_06_20_17_Q-Exactive-HF-X-Orbitrap_6070 28.167    NaN     NaN 30.170   
2020_01_08_16_43_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN  24.496 28.074   
2020_01_09_11_07_Q-Exactive-HF-X-Orbitrap_6070 25.986    NaN     NaN 28.587   
2020_01_15_13_56_Q-Exactive-HF-X-Orbitrap_6070 26.254 21.824     NaN    NaN   
2020_01_20_15_10_Q-Exactive-HF-X-Orbitrap_6070 28.211 26.320  25.625 30.534   
2020_02_05_20_55_Q-Exactive-HF-X-Orbitrap_6070 27.077 24.161  24.677 30.012   
2020_02_10_15_41_Q-Exactive-HF-X-Orbitrap_6070 28.929 27.426  26.594 30.291   
2020_02_11_10_35_Q-Exactive-HF-X-Orbitrap_6070    NaN 27.732     NaN 29.475   
2020_02_12_05_06_Q-Exactive-HF-X-Orbitrap_6070 28.755 27.338  25.054 30.123   
2020_02_13_00_26_Q-Exactive-HF-X-Orbitrap_6070 27.898 27.007     NaN 30.262   
2020_02_13_03_11_Q-Exactive-HF-X-Orbitrap_6070 28.006    NaN     NaN 30.520   
2020_02_17_13_55_Q-Exactive-HF-X-Orbitrap_6070 26.224 26.225     NaN    NaN   
2020_02_18_01_25_Q-Exactive-HF-X-Orbitrap_6070 23.969 25.867  24.077 28.756   
2020_02_18_18_55_Q-Exactive-HF-X-Orbitrap_6070 27.353 25.847  26.323 28.709   
2020_02_28_12_27_Q-Exactive-HF-X-Orbitrap_6070    NaN 25.666  25.250 28.246   
2020_03_01_23_00_Q-Exactive-HF-X-Orbitrap_6070 26.878 24.854  25.076    NaN   
2020_03_06_16_22_Q-Exactive-HF-X-Orbitrap_6070 24.384    NaN  22.665 27.200   
2020_03_07_18_15_Q-Exactive-HF-X-Orbitrap_6070 26.882 23.449  25.003    NaN   
2020_03_11_11_25_Q-Exactive-HF-X-Orbitrap_6070 27.523 26.516  25.779 28.829   
2020_05_04_11_39_Q-Exactive-HF-X-Orbitrap_6070 25.236 23.024  24.088 27.276   
2020_05_12_15_13_Q-Exactive-HF-X-Orbitrap_6070 26.513 24.702  22.879 27.572   
2020_05_12_18_10_Q-Exactive-HF-X-Orbitrap_6070 25.656 24.864  23.389 26.715   
2020_05_12_21_07_Q-Exactive-HF-X-Orbitrap_6070    NaN 24.781     NaN 27.428   
2020_05_14_14_46_Q-Exactive-HF-X-Orbitrap_6070 26.838 23.192  23.000 27.728   
2020_05_14_17_28_Q-Exactive-HF-X-Orbitrap_6070 26.415    NaN     NaN 27.776   
2020_05_14_20_11_Q-Exactive-HF-X-Orbitrap_6070 26.066 23.378  24.712 27.678   
2020_05_15_10_30_Q-Exactive-HF-X-Orbitrap_6070 24.857 26.531  24.117 28.240   
2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070 25.893    NaN     NaN 28.876   
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070 26.726 25.641     NaN    NaN   
2020_05_22_14_57_Q-Exactive-HF-X-Orbitrap_6070 25.693 24.134  25.063 28.467   
2020_05_22_17_43_Q-Exactive-HF-X-Orbitrap_6070 25.536 24.688     NaN 28.375   
2020_05_26_14_20_Q-Exactive-HF-X-Orbitrap_6070 27.263 26.631  25.491 28.849   
2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070 28.294 27.258  27.386 30.671   
2020_05_28_04_06_Q-Exactive-HF-X-Orbitrap_6070 28.023 27.669     NaN 30.908   
2020_06_01_10_22_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN 27.885   
2020_06_01_15_41_Q-Exactive-HF-X-Orbitrap_6070    

In [17]:
data.val_y  # potentially has less features

Gene Names                                       AAR2  ABCB6  ABHD11  ACAD9  \
Sample ID                                                                     
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2019_12_29_18_18_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_01_02_17_38_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_01_03_20_10_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_01_04_10_03_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_01_06_20_17_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_01_08_16_43_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_01_09_11_07_Q-Exactive-HF-X-Orbitrap_6070    NaN 31.639     NaN    NaN   
2020_01_15_13_56_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_01_20_15_10_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_02_05_20_55_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_02_10_15_41_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_02_11_10_35_Q-Exactive-HF-X-Orbitrap_6070 28.682    NaN  26.741    NaN   
2020_02_12_05_06_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_02_13_00_26_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_02_13_03_11_Q-Exactive-HF-X-Orbitrap_6070    NaN 27.599     NaN    NaN   
2020_02_17_13_55_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN  25.820 28.445   
2020_02_18_01_25_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_02_18_18_55_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_02_28_12_27_Q-Exactive-HF-X-Orbitrap_6070 27.248    NaN     NaN    NaN   
2020_03_01_23_00_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_03_06_16_22_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_03_07_18_15_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_03_11_11_25_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_05_04_11_39_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_05_12_15_13_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_05_12_18_10_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_05_12_21_07_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_05_14_14_46_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_05_14_17_28_Q-Exactive-HF-X-Orbitrap_6070    NaN 24.392     NaN    NaN   
2020_05_14_20_11_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_05_15_10_30_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN  24.352    NaN   
2020_05_22_14_57_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_05_22_17_43_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_05_26_14_20_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_05_28_04_06_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN  28.028    NaN   
2020_06_01_10_22_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN     NaN    NaN   
2020_06_01_15_41_Q-Exactive-HF-X-Orbitrap_6070    

In [18]:
data.val_y = pd.DataFrame(pd.NA, index=data.train_X.index,
                          columns=data.train_X.columns).fillna(data.val_y)
data.val_y

Gene Names                                       AAR2  ABCB6 ABHD11  ACAD9  \
Sample ID                                                                    
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_29_18_18_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_02_17_38_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_03_20_10_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_04_10_03_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_06_20_17_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_08_16_43_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_09_11_07_Q-Exactive-HF-X-Orbitrap_6070    NaN 31.639    NaN    NaN   
2020_01_15_13_56_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_20_15_10_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_05_20_55_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_10_15_41_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_11_10_35_Q-Exactive-HF-X-Orbitrap_6070 28.682    NaN 26.741    NaN   
2020_02_12_05_06_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_13_00_26_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_13_03_11_Q-Exactive-HF-X-Orbitrap_6070    NaN 27.599    NaN    NaN   
2020_02_17_13_55_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 25.820 28.445   
2020_02_18_01_25_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_18_18_55_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_28_12_27_Q-Exactive-HF-X-Orbitrap_6070 27.248    NaN    NaN    NaN   
2020_03_01_23_00_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_03_06_16_22_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_03_07_18_15_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_03_11_11_25_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_04_11_39_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_12_15_13_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_12_18_10_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_12_21_07_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_14_14_46_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_14_17_28_Q-Exactive-HF-X-Orbitrap_6070    NaN 24.392    NaN    NaN   
2020_05_14_20_11_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_15_10_30_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 24.352    NaN   
2020_05_22_14_57_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_22_17_43_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_26_14_20_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_28_04_06_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 28.028    NaN   
2020_06_01_10_22_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_06_01_15_41_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN 29.455   
2020_06_02_09_41_Q-Exa

## Variational Autoencoder

### Analysis: DataLoaders, Model, transform

In [19]:
default_pipeline = sklearn.pipeline.Pipeline(
    [
        ('normalize', StandardScaler()),
        ('impute', SimpleImputer(add_indicator=False))
    ])

### Analysis: DataLoaders, Model

In [20]:

analysis = ae.AutoEncoderAnalysis(  # datasplits=data,
    train_df=data.train_X,
    val_df=data.val_y,
    model=models.vae.VAE,
    model_kwargs=dict(n_features=data.train_X.shape[-1],
                      n_neurons=args.hidden_layers,
                      # last_encoder_activation=None,
                      last_decoder_activation=None,
                      dim_latent=args.latent_dim),
    transform=default_pipeline,
    decode=['normalize'],
    bs=args.batch_size)
args.n_params = analysis.n_params_ae
if args.cuda:
    analysis.model = analysis.model.cuda()
analysis.model

VAE(
  (encoder): Sequential(
    (0): Linear(in_features=376, out_features=512, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): LeakyReLU(negative_slope=0.1)
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): Dropout(p=0.2, inplace=False)
    (6): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.1)
    (8): Linear(in_features=256, out_features=50, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=25, out_features=256, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): LeakyReLU(negative_slope=0.1)
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): Dropout(p=0.2, inplace=False)
    (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


### Training



In [21]:
results = []
loss_fct = partial(models.vae.loss_fct, results=results)

In [22]:
analysis.learn = Learner(dls=analysis.dls,
                         model=analysis.model,
                         loss_func=loss_fct,
                         cbs=[ae.ModelAdapterVAE(),
                              EarlyStoppingCallback(patience=args.patience)
                              ])

analysis.learn.show_training_loop()

Start Fit
   - before_fit     : [TrainEvalCallback, Recorder, ProgressCallback, EarlyStoppingCallback]
  Start Epoch Loop
     - before_epoch   : [Recorder, ProgressCallback]
    Start Train
       - before_train   : [TrainEvalCallback, Recorder, ProgressCallback]
      Start Batch Loop
         - before_batch   : [ModelAdapterVAE, CastToTensor]
         - after_pred     : [ModelAdapterVAE]
         - after_loss     : [ModelAdapterVAE]
         - before_backward: []
         - before_step    : []
         - after_step     : []
         - after_cancel_batch: []
         - after_batch    : [TrainEvalCallback, Recorder, ProgressCallback]
      End Batch Loop
    End Train
     - after_cancel_train: [Recorder]
     - after_train    : [Recorder, ProgressCallback]
    Start Valid
       - before_validate: [TrainEvalCallback, Recorder, ProgressCallback]
      Start Batch Loop
         - **CBs same as train batch**: []
      End Batch Loop
    End Valid
     - after_cancel_validate: [Recorder]

Adding a `EarlyStoppingCallback` results in an error.  Potential fix in
[PR3509](https://github.com/fastai/fastai/pull/3509) is not yet in
current version. Try again later

In [23]:
# learn.summary()

In [24]:
suggested_lr = analysis.learn.lr_find()
analysis.params['suggested_inital_lr'] = suggested_lr.valley
suggested_lr

SuggestedLRs(valley=0.0008317637839354575)

In [25]:
results.clear()  # reset results

dump model config

In [26]:
# needs class as argument, not instance, but serialization needs instance
analysis.params['last_decoder_activation'] = Sigmoid()

vaep.io.dump_json(
    vaep.io.parse_dict(
        analysis.params, types=[
            (torch.nn.modules.module.Module, lambda m: str(m))
        ]),
    args.out_models / TEMPLATE_MODEL_PARAMS.format(args.model_key))

# restore original value
analysis.params['last_decoder_activation'] = Sigmoid

In [27]:
# papermill_description=train
analysis.learn.fit_one_cycle(args.epochs_max, lr_max=suggested_lr.valley)

Save number of actually trained epochs

In [28]:
args.epoch_trained = analysis.learn.epoch + 1
args.epoch_trained

5

#### Loss normalized by total number of measurements

In [29]:
N_train_notna = data.train_X.notna().sum().sum()
N_val_notna = data.val_y.notna().sum().sum()
fig = models.plot_training_losses(analysis.learn, args.model_key,
                                  folder=args.out_figures,
                                  norm_factors=[N_train_notna, N_val_notna])

vaep.plotting - INFO     Saved Figures to runs/example/figures/vae_training


### Predictions
create predictions and select validation data predictions

In [30]:
analysis.model.eval()
pred, target = res = ae.get_preds_from_df(df=data.train_X, learn=analysis.learn,
                                          position_pred_tuple=0,
                                          transformer=analysis.transform)
pred = pred.stack()
pred

Sample ID                                       Gene Names
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070  AAR2         26.593
                                                ABCB6        24.767
                                                ABHD11       24.783
                                                ACAD9        28.555
                                                ACTL6A       29.457
                                                              ...  
2020_06_02_09_41_Q-Exactive-HF-X-Orbitrap_6070  WNK1         28.283
                                                YBX1         32.545
                                                YTHDF1       26.565
                                                YWHAH        30.531
                                                ZNF326       28.773
Length: 18800, dtype: float32

In [31]:
val_pred_simulated_na['VAE'] = pred  # 'model_key' ?
val_pred_simulated_na

observed    VAE
Sample ID                                      Gene Names                 
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 GEMIN5        30.946 30.223
2020_02_28_12_27_Q-Exactive-HF-X-Orbitrap_6070 AAR2          27.248 26.709
2020_02_13_03_11_Q-Exactive-HF-X-Orbitrap_6070 ADAM10        29.310 26.128
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070 NDUFAF4       24.352 27.741
2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070 EPN1          29.250 27.502
...                                                             ...    ...
2020_03_01_23_00_Q-Exactive-HF-X-Orbitrap_6070 PRRC2B        24.294 25.445
2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070 ARGLU1        28.471 28.170
2020_02_18_18_55_Q-Exactive-HF-X-Orbitrap_6070 GBF1          29.470 28.600
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 DENR          27.902 26.836
2020_05_12_21_07_Q-Exactive-HF-X-Orbitrap_6070 SNRPF         27.487 29.164

[859 rows x 2 columns]

In [32]:
test_pred_simulated_na['VAE'] = pred  # model_key?
test_pred_simulated_na

observed    VAE
Sample ID                                      Gene Names                 
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 ARAF          23.248 26.009
                                               CENPH         25.045 24.966
                                               CLPP          27.745 27.651
                                               GNAI1         24.335 26.203
                                               KIAA1143      26.901 26.400
...                                                             ...    ...
2020_06_02_09_41_Q-Exactive-HF-X-Orbitrap_6070 NUDT9         27.658 26.677
                                               PTPN1         29.024 28.667
                                               RDH13         28.720 27.306
                                               RNASEH2C      26.742 25.991
                                               SH3BGRL3      29.162 29.295

[859 rows x 2 columns]

save missing values predictions

In [33]:
if args.save_pred_real_na:
    pred_real_na = ae.get_missing_values(df_train_wide=data.train_X,
                                         val_idx=val_pred_simulated_na.index,
                                         test_idx=test_pred_simulated_na.index,
                                         pred=pred)
    display(pred_real_na)
    pred_real_na.to_csv(args.out_preds / f"pred_real_na_{args.model_key}.csv")

Sample ID                                       Gene Names 
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070  ARMCX3        24.838
                                                BDH2          24.776
                                                CCBL2         26.242
                                                COMMD5        25.184
                                                DNAJC10       26.174
                                                               ...  
2020_06_02_09_41_Q-Exactive-HF-X-Orbitrap_6070  TMEM261       25.795
                                                TRMT2A        26.110
                                                TSNAX;DISC1   27.505
                                                UTRN          26.788
                                                VPS11         26.059
Name: intensity, Length: 1616, dtype: float32

### Plots

- validation data

In [34]:
analysis.model = analysis.model.cpu()
df_latent = vaep.model.get_latent_space(analysis.model.get_mu_and_logvar,
                                        dl=analysis.dls.valid,
                                        dl_index=analysis.dls.valid.data.index)
df_latent

latent dimension 1  \
Sample ID                                                            
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070              -0.056   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070               0.048   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070               0.045   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070               0.171   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070              -0.209   
2019_12_29_18_18_Q-Exactive-HF-X-Orbitrap_6070              -0.315   
2020_01_02_17_38_Q-Exactive-HF-X-Orbitrap_6070              -0.324   
2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070              -0.279   
2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070              -0.242   
2020_01_03_20_10_Q-Exactive-HF-X-Orbitrap_6070              -0.537   
2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070              -0.142   
2020_01_04_10_03_Q-Exactive-HF-X-Orbitrap_6070               0.205   
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070              -0.366   
2020_01_06_20_17_Q-Exactive-HF-X-Orbitrap_6070              -0.375   
2020_01_08_16_43_Q-Exactive-HF-X-Orbitrap_6070              -0.586   
2020_01_09_11_07_Q-Exactive-HF-X-Orbitrap_6070              -0.167   
2020_01_15_13_56_Q-Exactive-HF-X-Orbitrap_6070              -0.299   
2020_01_20_15_10_Q-Exactive-HF-X-Orbitrap_6070              -0.207   
2020_02_05_20_55_Q-Exactive-HF-X-Orbitrap_6070              -0.116   
2020_02_10_15_41_Q-Exactive-HF-X-Orbitrap_6070               0.190   
2020_02_11_10_35_Q-Exactive-HF-X-Orbitrap_6070              -0.058   
2020_02_12_05_06_Q-Exactive-HF-X-Orbitrap_6070              -0.050   
2020_02_13_00_26_Q-Exactive-HF-X-Orbitrap_6070              -0.052   
2020_02_13_03_11_Q-Exactive-HF-X-Orbitrap_6070               0.079   
2020_02_17_13_55_Q-Exactive-HF-X-Orbitrap_6070               0.044   
2020_02_18_01_25_Q-Exactive-HF-X-Orbitrap_6070               0.157   
2020_02_18_18_55_Q-Exactive-HF-X-Orbitrap_6070               0.121   
2020_02_28_12_27_Q-Exactive-HF-X-Orbitrap_6070               0.019   
2020_03_01_23_00_Q-Exactive-HF-X-Orbitrap_6070              -0.020   
2020_03_06_16_22_Q-Exactive-HF-X-Orbitrap_6070              -0.342   
2020_03_07_18_15_Q-Exactive-HF-X-Orbitrap_6070              -0.459   
2020_03_11_11_25_Q-Exactive-HF-X-Orbitrap_6070               0.094   
2020_05_04_11_39_Q-Exactive-HF-X-Orbitrap_6070               0.023   
2020_05_12_15_13_Q-Exactive-HF-X-Orbitrap_6070               0.113   
2020_05_12_18_10_Q-Exactive-HF-X-Orbitrap_6070               0.132   
2020_05_12_21_07_Q-Exactive-HF-X-Orbitrap_6070              -0.050   
2020_05_14_14_46_Q-Exactive-HF-X-Orbitrap_6070              -0.008   
2020_05_14_17_28_Q-Exactive-HF-X-Orbitrap_6070               0.198   
2020_05_14_20_11_Q-Exactive-HF-X-Orbitrap_6070               0.148   
2020_05_15_10_30_Q-Exactive-HF-X-Orbitrap_6070               0.097   
2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070               0.032   
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070              -0.044   
2020_05_22_14_57_Q-Exactive-HF-X-Orbitrap_6070               0.368   
2020_05_22_17_43_Q-Exactive-HF-X-Orbitrap_6070               0.276   
2020_05_26_14_20_Q-Exactive-HF-X-Orbitrap_6070               0.080   
2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070               0.107   
2020_05_28_04_06_Q-Exactive-HF-X-Orbitrap_6070              -0.428   
2020_06_01_10_22_Q-Exactive-HF-X-Orbitrap_6070              -0.487   
2020_06_01_15_41_Q-Exactive-HF-X-Orbitrap_6070               0.043   
2020_06_02_09_41_Q-Exactive-HF-X-Orbitrap_6070               0.043   

                                                latent dimension 2  \
Sample ID                                                            
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070              -0.128   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070              -0.348   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070               0.438   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070           

In [35]:
ana_latent = analyzers.LatentAnalysis(df_latent,
                                      df_meta,
                                      args.model_key,
                                      folder=args.out_figures)
if args.meta_date_col and df_meta is not None:
    figures[f'latent_{args.model_key}_by_date'], ax = ana_latent.plot_by_date(
        args.meta_date_col)

In [36]:
if args.meta_cat_col and df_meta is not None:
    figures[f'latent_{args.model_key}_by_{"_".join(args.meta_cat_col.split())}'], ax = ana_latent.plot_by_category(
        args.meta_cat_col)

In [37]:
feat_freq_val = val_pred_simulated_na['observed'].groupby(level=-1).count()
feat_freq_val.name = 'freq_val'
ax = feat_freq_val.plot.box()

In [38]:
feat_freq_val.value_counts().sort_index().head()  # require more than one feat?

1   101
2    85
3    89
4    41
5    16
Name: freq_val, dtype: int64

In [39]:
errors_val = val_pred_simulated_na.drop('observed', axis=1).sub(
    val_pred_simulated_na['observed'], axis=0)
errors_val = errors_val.abs().groupby(level=-1).mean()
errors_val = errors_val.join(freq_feat).sort_values(by='freq', ascending=True)


errors_val_smoothed = errors_val.copy()  # .loc[feat_freq_val > 1]
errors_val_smoothed[errors_val.columns[:-1]] = errors_val[errors_val.columns[:-1]
                                                          ].rolling(window=200, min_periods=1).mean()
ax = errors_val_smoothed.plot(x='freq', figsize=(15, 10))
# errors_val_smoothed

In [40]:
errors_val = val_pred_simulated_na.drop('observed', axis=1).sub(
    val_pred_simulated_na['observed'], axis=0)
errors_val.abs().groupby(level=-1).agg(['mean', 'count'])

VAE      
            mean count
Gene Names            
AAR2       0.942     2
ABCB6      2.693     3
ABHD11     1.377     4
ACAD9      0.299     2
ACTL6A     1.361     2
...          ...   ...
WNK1       0.680     3
YBX1       0.450     3
YTHDF1     1.242     2
YWHAH      0.558     1
ZNF326     1.234     3

[344 rows x 2 columns]

In [41]:
errors_val

VAE
Sample ID                                      Gene Names       
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 GEMIN5     -0.722
2020_02_28_12_27_Q-Exactive-HF-X-Orbitrap_6070 AAR2       -0.539
2020_02_13_03_11_Q-Exactive-HF-X-Orbitrap_6070 ADAM10     -3.181
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070 NDUFAF4     3.389
2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070 EPN1       -1.748
...                                                          ...
2020_03_01_23_00_Q-Exactive-HF-X-Orbitrap_6070 PRRC2B      1.151
2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070 ARGLU1     -0.301
2020_02_18_18_55_Q-Exactive-HF-X-Orbitrap_6070 GBF1       -0.869
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 DENR       -1.066
2020_05_12_21_07_Q-Exactive-HF-X-Orbitrap_6070 SNRPF       1.677

[859 rows x 1 columns]

## Comparisons

Simulated NAs : Artificially created NAs. Some data was sampled and set
explicitly to misssing before it was fed to the model for
reconstruction.

### Validation data

- all measured (identified, observed) peptides in validation data

In [42]:
# papermill_description=metrics
# d_metrics = models.Metrics(no_na_key='NA interpolated', with_na_key='NA not interpolated')
d_metrics = models.Metrics()

The simulated NA for the validation step are real test data (not used for training nor early stopping)

In [43]:
added_metrics = d_metrics.add_metrics(val_pred_simulated_na, 'valid_simulated_na')
added_metrics

Selected as truth to compare to: observed


{'VAE': {'MSE': 1.4309945391204608,
  'MAE': 0.895814664488961,
  'N': 859,
  'prop': 1.0}}

### Test Datasplit


In [44]:
added_metrics = d_metrics.add_metrics(test_pred_simulated_na, 'test_simulated_na')
added_metrics

Selected as truth to compare to: observed


{'VAE': {'MSE': 1.3382427575564606,
  'MAE': 0.8998851898713764,
  'N': 859,
  'prop': 1.0}}

Save all metrics as json

In [45]:
vaep.io.dump_json(d_metrics.metrics, args.out_metrics /
                  f'metrics_{args.model_key}.json')
d_metrics

{ 'test_simulated_na': { 'VAE': { 'MAE': 0.8998851898713764,
                                  'MSE': 1.3382427575564606,
                                  'N': 859,
                                  'prop': 1.0}},
  'valid_simulated_na': { 'VAE': { 'MAE': 0.895814664488961,
                                   'MSE': 1.4309945391204608,
                                   'N': 859,
                                   'prop': 1.0}}}

In [46]:
metrics_df = models.get_df_from_nested_dict(
    d_metrics.metrics, column_levels=['model', 'metric_name']).T
metrics_df

subset             valid_simulated_na  test_simulated_na
model metric_name                                       
VAE   MSE                       1.431              1.338
      MAE                       0.896              0.900
      N                       859.000            859.000
      prop                      1.000              1.000

## Save predictions

In [47]:
# save simulated missing values for both splits
val_pred_simulated_na.to_csv(args.out_preds / f"pred_val_{args.model_key}.csv")
test_pred_simulated_na.to_csv(args.out_preds / f"pred_test_{args.model_key}.csv")

## Config

In [48]:
figures  # switch to fnames?

{}

In [49]:
args.dump(fname=args.out_models / f"model_config_{args.model_key}.yaml")
args

{'M': 376,
 'batch_size': 10,
 'cuda': False,
 'data': Path('runs/example/data'),
 'epoch_trained': 5,
 'epochs_max': 5,
 'file_format': 'csv',
 'fn_rawfile_metadata': 'data/dev_datasets/HeLa_6070/files_selected_metadata_N50.csv',
 'folder_data': '',
 'folder_experiment': Path('runs/example'),
 'hidden_layers': [512, 256],
 'latent_dim': 25,
 'meta_cat_col': None,
 'meta_date_col': None,
 'model': 'VAE',
 'model_key': 'VAE',
 'n_params': 864290,
 'out_figures': Path('runs/example/figures'),
 'out_folder': Path('runs/example'),
 'out_metrics': Path('runs/example'),
 'out_models': Path('runs/example'),
 'out_preds': Path('runs/example/preds'),
 'patience': 50,
 'sample_idx_position': 0,
 'save_pred_real_na': True}